In [17]:
from os import path
import numpy as np
import matplotlib.pyplot as plt
import sys
sys.path.append('/projects/patho1/melanoma_diagnosis/code/multi-scale_melanoma_diagnosis/')
from sklearn.metrics import confusion_matrix as c_matrix


In [100]:
results_dir = '/projects/patho1/melanoma_diagnosis/results/binarize/4scale_49_512x2x64'
results_txt = 'result.txt'

In [101]:
with open(path.join(results_dir, results_txt), 'r') as f:
    lines = [line.rstrip() for line in f]

In [102]:
confusion_matrix = np.zeros((4,4), dtype=int)
confidence_matrix = {0: {}, 1: {}, 2: {}, 3:{}}
confidence_case_matrix = {}
case_pred = {}
case_target = {}
case_lookup = {0: {}, 1: {}, 2: {}, 3:{}}
slice_pred = []
slice_label = []

In [103]:
for line in lines:
    line = line.split(',')
    im_p = line[0]
    pred = int(line[1])
    slice_pred.append(pred)
#     scores_raw = line[2].split(',')
#     scores = [float(scores_raw[i].replace('[', '').replace(']', '')) for i in range(len(scores_raw))]
    scores_raw = line[2:2+4]
    scores = [float(scores_raw[i].replace('[', '').replace(']', '')) for i in range(len(scores_raw))]
    label = max(0, int(path.basename(path.dirname(im_p))) - 2)
    slice_label.append(label)
    bn = path.basename(im_p)
    im_ind = path.splitext(bn)[0]
    case = im_ind.split('_x10')[0]
    confusion_matrix[label][pred] += 1
    if pred not in confidence_matrix[label]:
        confidence_matrix[label][pred] = [scores]
        case_lookup[label][pred] = [im_ind]
    else:
        confidence_matrix[label][pred].append(scores)
        case_lookup[label][pred].append(im_ind)
    if case not in confidence_case_matrix:
        confidence_case_matrix[case] = {} 
        case_pred[case] = pred
        case_target[case] = label
    else:
        if pred > case_pred[case]:
            case_pred[case] = pred
    confidence_case_matrix[case][str(im_ind[-1])] = scores

In [104]:
case_confusion_matrix = np.zeros((4,4))
pred_list = []
label_list = []
for case in case_pred:
    pred = case_pred[case]
    label = case_target[case]
    case_confusion_matrix[label][pred] += 1
    pred_list.append(pred)
    label_list.append(label)


In [105]:
from metrics.cmat_metrics import CMMetrics, CMResults
results_summary = dict()
results_summary['true_labels'] = [int(x) for x in label_list]
results_summary['pred_labels'] = [int(x) for x in pred_list]
#print(len(y_true))
#print(len(y_pred))
cmat = c_matrix(results_summary['true_labels'], results_summary['pred_labels'])
cmat_np_arr = np.array(cmat)
conf_mat_eval = CMMetrics()
cmat_results: CMResults = conf_mat_eval.compute_metrics(conf_mat=cmat_np_arr)
cmat_results_dict = cmat_results._asdict()
for k, v in cmat_results_dict.items():
    if isinstance(v, np.ndarray):
        values = v.tolist()
    results_summary['{}'.format(k)] = v

In [106]:
print(results_summary['overall_accuracy'])
print(results_summary['sensitivity_micro'])
print(results_summary['specificity_micro'])
print(results_summary['f1_micro'])

0.3873873873524876
0.3873873873524876
0.795795795771898
0.3873873873524876


In [107]:
print(case_confusion_matrix)

[[23.  3.  0.  3.]
 [22.  5.  1.  2.]
 [19.  3.  1.  7.]
 [ 5.  0.  3. 14.]]


In [88]:
import csv
csv_filename = path.join(results_dir, 'train_case.csv')
fields = ['case ID', 'Prediction', 'Label', 'slice number', '0', '1', '2', '3']
with open(csv_filename, 'w') as csv_f:
    csvwriter = csv.writer(csv_f)
    csvwriter.writerow(fields) 
    for case in confidence_case_matrix:
        row = [case]
        row.append(str(case_pred[case]))
        row.append(str(case_target[case]))
        csvwriter.writerow(row)
        for i in confidence_case_matrix[case]:
            slice_row = ['', '', '']
            slice_row.append(str(i))
            for conf in confidence_case_matrix[case][str(i)]:
                slice_row.append(str(conf))
            csvwriter.writerow(slice_row)


In [ ]:
fig, ax = plt.subplots(nrows=2, ncols=2, figsize=(15,10))
for j in range(4):
    class_conf = [x[j] for x in confidence_matrix[1][2]]
    row = j//2
    col = j-2*(j//2)
    _ = ax[row][col].hist(class_conf, bins=[0, 0.25, 0.5, 0.75, 1])
    ax[row][col].title.set_text('class {}'.format(j))
plt.setp(ax, xticks=[0, 0.25, 0.5, 0.75, 1], yticks=range(0, 36, 2))


In [ ]:
print(confidence_case_matrix['MP_0052'])